In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import glob,re
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
param_pattern = re.compile(r'\d\d-\d\d-\d\d \d{6}_(?P<type>sqr|tri|sqr|saw)_(?P<freq>\d+.?\d*)_(?P<amp>\d+.?\d*).*')
def getData(subpat):
    d = {}
    for f in glob.glob(subpat):
        result = param_pattern.findall(f)[0]
        (typ,freq,amp) = result
        freq,amp = float(freq),float(amp)
        if(typ not in d.keys()):
            d[typ]= {}
        if(freq not in d[typ].keys()):
            d[typ][freq] = {}
        if(amp not in d[typ][freq].keys()):
            d[typ][freq][amp] = []
        data = pd.read_csv(f)
        if(data.shape[0] > 0 and data.shape[1] > 0):
            d[typ][freq][amp].append(data)
    return d

In [3]:
def calcAveAndStd(data):
    n = len(data)
    
    ave = data[0].copy()
    for d in data[1:]:
        ave += d.copy()
    
    ave = ave / n
    
    std = (ave - data[0].copy())**2
    for d in data[1:]:
        std += (ave - d.copy())**2
    
    std = (std / n).apply(np.sqrt)
    std['Generation'] = ave.copy()['Generation']
    return ave,std

In [4]:
def calcAllStats(data):
    aves,stds = {},{}

    for typ in data.keys():
        aves[typ],stds[typ] = {},{}
        for freq in data[typ]:
            aves[typ][freq],stds[typ][freq] = {},{}
            for amp in data[typ][freq]:
                aves[typ][freq][amp],stds[typ][freq][amp] = calcAveAndStd(data[typ][freq][amp])
    return aves,stds

In [5]:
def gaTrace(data,error,sortedKeys = None,ci_multipler = 1.95,fig=None,ax=None):
    
    if(sortedKeys is None):
        sortedKeys = sorted(data.keys())
    
    n = len(sortedKeys)
    
    seaborn.set_style('white')
    seaborn.set_context("paper", font_scale=2, rc={"lines.linewidth": 1.5})
    # colors = seaborn.cubehelix_palette(7, start=1, rot=-5,dark=.1, light=.5)
    colors = seaborn.husl_palette(n,l=.4)
    shaded = np.hstack([colors,0.25*np.ones((n,1))])
    
    if(fig is None or ax is None):
        fig,ax = plt.subplots(figsize=(15,9))
    
    for i,k in enumerate(sortedKeys):
        u = data[k]['Max fitness']
        ci = error[k]['Max fitness']*ci_multipler

        ax.plot(u,color=colors[i],label=k)
        ax.fill_between(data[k]['Generation']-1,u-ci,u+ci,color=shaded[i])

    return fig,ax

In [6]:
def drawAllGAPlots_amps(aves,stds):
    for typ in aves.keys():
        for freq in aves[typ].keys():
            fig,ax = gaTrace(aves[typ][freq],stds[typ][freq])

            plt.title("%s    %s" % (typ, freq))
            plt.ylabel('Maximum fitness')
            plt.xlabel('Generation')
            plt.legend(loc='lower right')
            fig.show()

In [7]:
def getFreqSummary(aves,stds):
    new_aves,new_stds = {},{}
    for typ in aves.keys():
        new_aves[typ],new_stds[typ] = {},{}
        for freq in aves[typ]:
            keys = list(aves[typ][freq].keys())
            
            new_aves[typ][freq] = aves[typ][freq][keys[0]].copy()
            new_stds[typ][freq] = stds[typ][freq][keys[0]].copy()
            for k in keys[1:]:
                new_aves[typ][freq] += aves[typ][freq][k].copy()
                new_stds[typ][freq] += stds[typ][freq][k].copy()
                
            new_aves[typ][freq] /= len(keys)
            new_stds[typ][freq] /= len(keys)
            
    return new_aves,new_stds

def drawAllGAPlots_freqs(aves,stds):
    for typ in aves.keys():
        fig,ax = gaTrace(aves[typ],stds[typ])

        plt.title(typ)
        plt.ylabel('Maximum fitness')
        plt.xlabel('Generation')
        plt.legend(loc='lower right')
        fig.show()

IndexError: list index out of range